In [1]:
import pandas as pd
import numpy as np
import torch
import gc
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
from sklearn import tree
from sklearn.model_selection import cross_val_score
from pymop import Problem
import os
from tslearn.datasets import UCR_UEA_datasets
import warnings
warnings.filterwarnings('ignore')


In [2]:
dataset='GunPoint'
X_train,train_y,X_test,test_y=UCR_UEA_datasets().load_dataset(dataset)
train_x=X_train.reshape(-1,X_train.shape[-1],X_train.shape[-2])
test_x=X_test.reshape(-1,X_train.shape[-1],X_train.shape[-2])


In [11]:
from ClassificationModels.CNN_T import ResNetBaseline, UCRDataset, get_all_preds
stride = 1
kernel_size=10
padding = kernel_size - 1
input_size= train_x.shape[-1]
device = torch.device( "cpu")#"cuda:0" if torch.cuda.is_available() else
model = ResNetBaseline(in_channels= 1, num_pred_classes=2)
model.load_state_dict(torch.load(f'./ClassificationModels/models/GunPoint/ResNet'))
test_dataset = UCRDataset(test_x,test_y)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=True)


In [12]:
model.eval()
y_pred= get_all_preds(model,test_loader)
#y_pred= model(test_x).detach().numpy()

In [13]:
observation_01, label_01 = test_dataset[0]
label_01=np.array([y_pred[0]])
labels=np.argmax(y_pred,axis=1)


In [18]:
from TSInterpret.InterpretabilityModels.counterfactual.TSEvoCF import TSEvo
exp =TSEvo(model= model,data=(test_x,np.argmax(y_pred,axis=1)), backend='PYT')

ModuleNotFoundError: No module named 'TSInterpret.InterpretabilityModels.Counterfactual'

In [ ]:

#TODO Call Explain Instead
pop,logbook=exp.explain_instance(observation_01,label_01,transformer = 'mutate_both')